<a href="https://colab.research.google.com/github/talhaanwarch/skin_lesion_diagnosis_app/blob/master/skin_ag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [1]:
%%capture
!python3 -m pip install -U pip
!python3 -m pip install -U setuptools wheel
!python3 -m pip install -U "mxnet_cu110<2.0.0"
!git clone https://github.com/awslabs/autogluon
!cd autogluon && ./full_install.sh

In [2]:
!wget https://md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com/zr7vgbcyr2-1.zip -O data.zip

--2021-06-17 22:46:28--  https://md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com/zr7vgbcyr2-1.zip
Resolving md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com (md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com)... 52.218.65.3
Connecting to md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com (md-datasets-cache-zipfiles-prod.s3.eu-west-1.amazonaws.com)|52.218.65.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3593344681 (3.3G) [application/octet-stream]
Saving to: ‘data.zip’

data.zip            100%[===================>]   3.35G  33.1MB/s    in 1m 51s  

2021-06-17 22:48:19 (30.8 MB/s) - ‘data.zip’ saved [3593344681/3593344681]



In [6]:
%%capture
import os
if os.path.isdir('data') is False:
  !mkdir data
  !unzip /content/data.zip -d data
  !unzip /content/data/images/imgs_part_1.zip -d data
  !unzip /content/data/images/imgs_part_2.zip -d data
  !unzip /content/data/images/imgs_part_3.zip -d data

  !mv  /content/data/imgs_part_1/* /content/data
  !mv  /content/data/imgs_part_2/* /content/data
  !mv  /content/data/imgs_part_3/* /content/data

  !rm -r /content/data/imgs_part_1
  !rm -r /content/data/imgs_part_2
  !rm -r /content/data/imgs_part_3
  !rm -r /content/data/images
  !rm -rf /content/data/__MACOSX

In [7]:
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
import cv2
import autogluon.core as ag
from autogluon.vision import ImagePredictor
from sklearn.metrics import classification_report

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data=pd.read_csv('/content/data/metadata.csv')
data.head()

,patient_id,lesion_id,smoke,drink,background_father,background_mother,age,pesticide,gender,skin_cancer_history,cancer_history,has_piped_water,has_sewage_system,fitspatrick,region,diameter_1,diameter_2,diagnostic,itch,grew,hurt,changed,bleed,elevation,img_id,biopsed
0,PAT_1516,1765,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ARM,NaN,NaN,NEV,False,False,False,False,False,False,PAT_1516_1765_530.png,False
1,PAT_46,881,False,False,POMERANIA,POMERANIA,55,False,FEMALE,True,True,True,True,3.0,NECK,6.0,5.0,BCC,True,True,False,True,True,True,PAT_46_881_939.png,True
2,PAT_1545,1867,NaN,NaN,NaN,NaN,77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FACE,NaN,NaN,ACK,True,False,False,False,False,False,PAT_1545_1867_547.png,False
3,PAT_1989,4061,NaN,NaN,NaN,NaN,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HAND,NaN,NaN,ACK,True,False,False,False,False,False,PAT_1989_4061_934.png,False
4,PAT_684,1302,False,True,POMERANIA,POMERANIA,79,False,MALE,True,False,False,False,1.0,FOREARM,5.0,5.0,BCC,True,True,False,False,True,True,PAT_684_1302_588.png,True


In [9]:
df=data[['img_id','diagnostic']].copy()
df.img_id='data/'+df.img_id.astype(str)
df.columns=['image','label']
df.head()

,image,label
0,data/PAT_1516_1765_530.png,NEV
1,data/PAT_46_881_939.png,BCC
2,data/PAT_1545_1867_547.png,ACK
3,data/PAT_1989_4061_934.png,ACK
4,data/PAT_684_1302_588.png,BCC


In [10]:
train_df,val_df=train_test_split(df,test_size=0.1,random_state=42)
train_df,test_df=train_test_split(df,test_size=0.15,random_state=42)

In [11]:
encode={k:v for v,k in enumerate(train_df.label.unique())}
decode={v:k for v,k in enumerate(train_df.label.unique())}

In [12]:
train_df['label']=train_df['label'].map(encode)
val_df.loc[:,'label']=val_df.loc[:,'label'].map(encode)
test_df['label']=test_df['label'].map(encode)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

In [16]:
pip show autogluon

Name: autogluon
Version: 0.2.1b20210617
Summary: AutoML for Text, Image, and Tabular Data
Home-page: https://github.com/awslabs/autogluon
Author: AutoGluon Community
Author-email: None
License: Apache-2.0
Location: /content/autogluon/autogluon/src
Requires: autogluon.core, autogluon.features, autogluon.tabular, autogluon.mxnet, autogluon.extra, autogluon.text, autogluon.vision
Required-by: 


In [ ]:
%%time
predictor = ImagePredictor(verbosity=1)
predictor.fit(train_df,tuning_data=val_df)

The number of requested GPUs is greater than the number of available GPUs.Reduce the number to 1


In [ ]:
predictor.save('/content/drive/MyDrive/skin.ag')
